# Simulate the striatum example network

This example continues ```striatum_example.ipynb``` and shows how to setup input and run the simulation.

## Setup the input

Here we use the file ```input_config/striatum-test-input.json```.

The input can be generated in serial or in parallel. To generate it in parallel, pass ```rc``` parameter as in the previous examples.

In [2]:
import os
from snudda.input import SnuddaInput

network_path = os.path.join("networks", "striatum_example")

si = SnuddaInput(network_path=network_path, 
                 input_config_file=os.path.join("input_config","striatum-test-input.json"),
                 verbose=False)
si.generate()

Writing spikes to networks/striatum_example/input-spikes.hdf5


## Compile the  NEURON mod files
If you run this on Windows you need to exchange / for \ in the path below.

In [ ]:
!nrnivmodl ../../snudda/data/neurons/mechanisms

## Run the neuron simulation

Here we run the network for 3.5 seconds.

In [3]:
!mpiexec -n 6 snudda simulate networks/striatum_example --time 3.5

See ```striatum_example_plot.ipynb``` for information on plotting a spike raster.